En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import datetime

In [2]:
class Extraccion:

    def __init__(self):

        self = self
        
    def api_nacional(self, año_inicio, año_final, df_vacio):

        self.año_inicio = año_inicio
        self.año_final = año_final
        self.df_vacio = df_vacio

        for año in range(año_inicio, año_final):
            
            response = requests.get(f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year')

            response.status_code

            response.reason

            df_renovables = pd.json_normalize(response.json()['included'][0]['attributes']['values'])
            df_renovables ['tipo_energía'] = "Renovable"

            df_vacio = pd.concat([df_vacio, df_renovables], axis = 0)

            df_no_renovables = pd.json_normalize(response.json()['included'][1]['attributes']['values']) 
            df_no_renovables ['tipo_energía'] = "No_renovable"

            df_vacio = pd.concat([df_vacio, df_no_renovables], axis = 0)
            
        return df_vacio
    

    def api_ccaa(self,año_inicio,año_final, diccionario, df_vacio):

        self.diccionario = diccionario
        self.año_inicio = año_inicio
        self.año_final = año_final
        self.df_vacio = df_vacio

        for año in range(año_inicio,año_final):

            for k, v in diccionario.items():
                    
                response = requests.get(f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}')

                response.status_code
                
                response.reason
            
                if len(response.json()['included']) == 2:
                    renovables_ccaa = pd.json_normalize(response.json()['included'][0]['attributes']['values']) #renovables
                    renovables_ccaa ['tipo_energía'] = "Renovable"
                    renovables_ccaa ['comunidad'] = k 
                    renovables_ccaa ['id_comunidad'] = v
                
                    df_vacio = pd.concat([df_vacio, renovables_ccaa], axis = 0)

                    no_renovables_ccaa = pd.json_normalize(response.json()['included'][1]['attributes']['values']) #excepción
                    no_renovables_ccaa ['tipo_energía'] = "No_renovable"
                    no_renovables_ccaa ['comunidad'] = k 
                    no_renovables_ccaa ['id_comunidad'] = v

                    df_vacio = pd.concat([df_vacio, no_renovables_ccaa], axis = 0)

                else:
                    no_renovables_ccaa1 = pd.json_normalize(response.json()['included'][0]['attributes']['values']) #excepción
                    no_renovables_ccaa1 ['tipo_energía'] = "No_renovable"
                    no_renovables_ccaa1 ['comunidad'] = k 
                    no_renovables_ccaa1 ['id_comunidad'] = v

                    df_vacio = pd.concat([df_vacio, no_renovables_ccaa1], axis = 0)

        return df_vacio


In [3]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [4]:
df = pd.DataFrame()

In [5]:
api = Extraccion( )
api

In [6]:
df_nacional = api.api_nacional(2011, 2022, df)
df_nacional

,value,percentage,datetime,tipo_energía
0,8.666705e+07,0.310241,2011-01-01T00:00:00.000+01:00,Renovable
0,1.926868e+08,0.689759,2011-01-01T00:00:00.000+01:00,No_renovable
0,8.533872e+07,0.301424,2012-01-01T00:00:00.000+01:00,Renovable
0,1.977799e+08,0.698576,2012-01-01T00:00:00.000+01:00,No_renovable
0,1.097577e+08,0.401622,2013-01-01T00:00:00.000+01:00,Renovable
0,1.635286e+08,0.598378,2013-01-01T00:00:00.000+01:00,No_renovable
0,1.078761e+08,0.404771,2014-01-01T00:00:00.000+01:00,Renovable
0,1.586356e+08,0.595229,2014-01-01T00:00:00.000+01:00,No_renovable
0,9.408811e+07,0.351792,2015-01-01T00:00:00.000+01:00,Renovable
0,1.733657e+08,0.648208,2015-01-01T00:00:00.000+01:00,No_renovable


In [7]:
help(Extraccion())

Help on Extraccion in module __main__ object:

class Extraccion(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  api_ccaa(self, año_inicio, año_final, diccionario, df_vacio)
 |  
 |  api_nacional(self, año_inicio, año_final, df_vacio)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [8]:
df2 = pd.DataFrame()

In [9]:
df_ccaa = api.api_ccaa(2011, 2022, cod_comunidades, df2)

In [10]:
df_ccaa.head()

,value,percentage,datetime,tipo_energía,comunidad,id_comunidad
0,2.029729e+05,1.000000,2011-01-01T00:00:00.000+01:00,No_renovable,Ceuta,8744
0,3.589082e+03,0.016703,2011-01-01T00:00:00.000+01:00,Renovable,Melilla,8745
0,2.112900e+05,0.983297,2011-01-01T00:00:00.000+01:00,No_renovable,Melilla,8745
0,1.099692e+07,0.284857,2011-01-01T00:00:00.000+01:00,Renovable,Andalucía,4
0,2.760811e+07,0.715143,2011-01-01T00:00:00.000+01:00,No_renovable,Andalucía,4


In [11]:
class Limpieza:
    
    def __init__(self, df):
        
        self= self
        self.df = df
    
    def redondeo(self, columna):

        self.columna = columna
        self.df[columna] = [round(float(values),2) for values in self.df[columna]]
        return self.df
    
    
    def eliminar_columna (self, columna):
        self.columna = columna

        self.df.drop([columna], axis= 1, inplace= True)
        return self.df
    
    def cambiar_datetime(self, columna):
        self.columna = columna

        self.df['fecha'] = self.df[columna].str.split("T", expand = True, n=1).get(0)
        self.df["fecha"] = pd.to_datetime(self.df["fecha"])
        self.df.drop([columna], axis = 1, inplace= True)
        return self.df
        

In [12]:
limpieza = Limpieza(df_nacional)

In [13]:
df_ccaa.dtypes

value           float64
percentage      float64
datetime         object
tipo_energía     object
comunidad        object
id_comunidad      int64
dtype: object

In [14]:
limpieza.redondeo('value')

,value,percentage,datetime,tipo_energía
0,8.666705e+07,0.310241,2011-01-01T00:00:00.000+01:00,Renovable
0,1.926868e+08,0.689759,2011-01-01T00:00:00.000+01:00,No_renovable
0,8.533872e+07,0.301424,2012-01-01T00:00:00.000+01:00,Renovable
0,1.977799e+08,0.698576,2012-01-01T00:00:00.000+01:00,No_renovable
0,1.097577e+08,0.401622,2013-01-01T00:00:00.000+01:00,Renovable
0,1.635286e+08,0.598378,2013-01-01T00:00:00.000+01:00,No_renovable
0,1.078761e+08,0.404771,2014-01-01T00:00:00.000+01:00,Renovable
0,1.586356e+08,0.595229,2014-01-01T00:00:00.000+01:00,No_renovable
0,9.408811e+07,0.351792,2015-01-01T00:00:00.000+01:00,Renovable
0,1.733657e+08,0.648208,2015-01-01T00:00:00.000+01:00,No_renovable


In [15]:

limpieza.redondeo('percentage')

,value,percentage,datetime,tipo_energía
0,8.666705e+07,0.31,2011-01-01T00:00:00.000+01:00,Renovable
0,1.926868e+08,0.69,2011-01-01T00:00:00.000+01:00,No_renovable
0,8.533872e+07,0.30,2012-01-01T00:00:00.000+01:00,Renovable
0,1.977799e+08,0.70,2012-01-01T00:00:00.000+01:00,No_renovable
0,1.097577e+08,0.40,2013-01-01T00:00:00.000+01:00,Renovable
0,1.635286e+08,0.60,2013-01-01T00:00:00.000+01:00,No_renovable
0,1.078761e+08,0.40,2014-01-01T00:00:00.000+01:00,Renovable
0,1.586356e+08,0.60,2014-01-01T00:00:00.000+01:00,No_renovable
0,9.408811e+07,0.35,2015-01-01T00:00:00.000+01:00,Renovable
0,1.733657e+08,0.65,2015-01-01T00:00:00.000+01:00,No_renovable


In [16]:
limpieza.cambiar_datetime('datetime')

,value,percentage,tipo_energía,fecha
0,8.666705e+07,0.31,Renovable,2011-01-01
0,1.926868e+08,0.69,No_renovable,2011-01-01
0,8.533872e+07,0.30,Renovable,2012-01-01
0,1.977799e+08,0.70,No_renovable,2012-01-01
0,1.097577e+08,0.40,Renovable,2013-01-01
0,1.635286e+08,0.60,No_renovable,2013-01-01
0,1.078761e+08,0.40,Renovable,2014-01-01
0,1.586356e+08,0.60,No_renovable,2014-01-01
0,9.408811e+07,0.35,Renovable,2015-01-01
0,1.733657e+08,0.65,No_renovable,2015-01-01


In [17]:
df_nacional.to_pickle('datos/df_nacional.pkl')

In [18]:
limpieza2 = Limpieza(df_ccaa)

In [19]:
limpieza2.redondeo('value')
limpieza2.redondeo('percentage')

,value,percentage,datetime,tipo_energía,comunidad,id_comunidad
0,202972.89,1.000000,2011-01-01T00:00:00.000+01:00,No_renovable,Ceuta,8744
0,3589.08,0.016703,2011-01-01T00:00:00.000+01:00,Renovable,Melilla,8745
0,211290.04,0.983297,2011-01-01T00:00:00.000+01:00,No_renovable,Melilla,8745
0,10996921.24,0.284857,2011-01-01T00:00:00.000+01:00,Renovable,Andalucía,4
0,27608105.11,0.715143,2011-01-01T00:00:00.000+01:00,No_renovable,Andalucía,4
...,...,...,...,...,...,...
0,6454252.22,0.799826,2021-01-01T00:00:00.000+01:00,No_renovable,Canarias,8742
0,2612050.26,0.282398,2021-01-01T00:00:00.000+01:00,Renovable,Región de Murcia,21
0,6637496.84,0.717602,2021-01-01T00:00:00.000+01:00,No_renovable,Región de Murcia,21
0,1198991.66,0.554635,2021-01-01T00:00:00.000+01:00,Renovable,La Rioja,20


,value,percentage,datetime,tipo_energía,comunidad,id_comunidad
0,202972.89,1.00,2011-01-01T00:00:00.000+01:00,No_renovable,Ceuta,8744
0,3589.08,0.02,2011-01-01T00:00:00.000+01:00,Renovable,Melilla,8745
0,211290.04,0.98,2011-01-01T00:00:00.000+01:00,No_renovable,Melilla,8745
0,10996921.24,0.28,2011-01-01T00:00:00.000+01:00,Renovable,Andalucía,4
0,27608105.11,0.72,2011-01-01T00:00:00.000+01:00,No_renovable,Andalucía,4
...,...,...,...,...,...,...
0,6454252.22,0.80,2021-01-01T00:00:00.000+01:00,No_renovable,Canarias,8742
0,2612050.26,0.28,2021-01-01T00:00:00.000+01:00,Renovable,Región de Murcia,21
0,6637496.84,0.72,2021-01-01T00:00:00.000+01:00,No_renovable,Región de Murcia,21
0,1198991.66,0.55,2021-01-01T00:00:00.000+01:00,Renovable,La Rioja,20


In [20]:
#limpieza2.eliminar_columna('comunidad')

In [21]:
limpieza2.cambiar_datetime('datetime')

,value,percentage,tipo_energía,comunidad,id_comunidad,fecha
0,202972.89,1.00,No_renovable,Ceuta,8744,2011-01-01
0,3589.08,0.02,Renovable,Melilla,8745,2011-01-01
0,211290.04,0.98,No_renovable,Melilla,8745,2011-01-01
0,10996921.24,0.28,Renovable,Andalucía,4,2011-01-01
0,27608105.11,0.72,No_renovable,Andalucía,4,2011-01-01
...,...,...,...,...,...,...
0,6454252.22,0.80,No_renovable,Canarias,8742,2021-01-01
0,2612050.26,0.28,Renovable,Región de Murcia,21,2021-01-01
0,6637496.84,0.72,No_renovable,Región de Murcia,21,2021-01-01
0,1198991.66,0.55,Renovable,La Rioja,20,2021-01-01


In [22]:
df_ccaa.to_pickle('datos/df_ccaa.pkl')